In [1]:
%load_ext autoreload
%autoreload 

import sys
import os
current = os.path.abspath('..')
sys.path.append(current)
from xarray_arr_mean import XarrayArrMean

In [2]:
files = [
    "s3://its-live-data/cloud-experiments/h5cloud/atl03/big/original/ATL03_20181120182818_08110112_006_02.h5",
    "s3://its-live-data/cloud-experiments/h5cloud/atl03/big/original/ATL03_20190219140808_08110212_006_02.h5",
]
test_original = XarrayArrMean('atl03-bigsize-original', files=files, store_results=False, anon_access=True)

In [ ]:
# don't even try this out of region.... more than 30 minutes 
io_params ={
    "fsspec_params": {},
    "h5py_params" : {}
}
test_original.run(io_params)

In [ ]:
files = [
    "s3://its-live-data/cloud-experiments/h5cloud/atl03/big/repacked/ATL03_20181120182818_08110112_006_02_repacked.h5",
    "s3://its-live-data/cloud-experiments/h5cloud/atl03/big/repacked/ATL03_20190219140808_08110212_006_02_repacked.h5",
]
test_cloud = XarrayArrMean('atl03-bigsize-repacked', files=files, store_results=False, anon_access=True)

In [ ]:
io_params ={
    "fsspec_params": {
        # "skip_instance_cache": True
        "cache_type": "blockcache",
        "block_size": 8*1024*1024
    },
    "h5py_params" : {
        "driver_kwds": {
            "page_buf_size": 32*1024*1024,
            "rdcc_nbytes": 8*1024*1024
        }

    }
}
test_cloud.run(io_params)